In [1]:
import pandas as pd
import re
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import load_model

# Load the dataset
file_path = "carlsen_games_moves.csv"  # Update this with your dataset file path
chess_data = pd.read_csv(file_path)

# Recreate Tokenizer
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(chess_data['move_sequence'])

# Recreate LabelEncoder
label_encoder = LabelEncoder()
label_encoder.fit(chess_data['notation'])

# Load the pre-trained Keras model
model = load_model('chess_tactics_chatbot.keras')

In [2]:
def chess_advisor(moves, model, tokenizer, label_encoder):
    """
    Predicts the next best move given a list of chess moves.
    """
    # Join moves to create a sequence
    move_sequence = '|'.join(moves)

    # Tokenize and pad the input sequence
    sequence = tokenizer.texts_to_sequences([move_sequence])
    sequence_padded = tf.keras.preprocessing.sequence.pad_sequences(
        sequence, maxlen=model.input_shape[1], padding='post'
    )

    # Predict the next move
    prediction = model.predict(sequence_padded)
    recommended_move = label_encoder.inverse_transform([tf.argmax(prediction[0]).numpy()])

    return recommended_move[0]

In [3]:
def interactive_chatbot(model, tokenizer, label_encoder):
    """Interactive chatbot for chess move prediction with dynamic input parsing."""
    print("Chatbot: Good day! I will try my best to predict chess moves based on your setting!")
    print("Type 'exit' to end the chat.")

    while True:
        user_input = input("User: ")
        if user_input.lower() == "exit":
            print("Chatbot: Thank you for chatting! Goodbye!")
            break

        # Extract moves from the user's input dynamically
        moves = extract_moves(user_input)
        if moves:
            print(f"Chatbot: Analyzing the moves: {moves}")
            # Predict the next move
            try:
                next_move = chess_advisor(moves, model, tokenizer, label_encoder)
                print(f"Chatbot: I can predict that the best next move is {next_move}.")
            except Exception as e:
                print(f"Chatbot: An error occurred: {e}")
        else:
            print("Chatbot: Sorry, I couldn't extract valid moves. Please provide a valid sequence of moves.")

In [4]:
def extract_moves(user_input):
    """Extract chess moves dynamically from user input."""
    # Updated regex to handle moves with punctuation and spaces
    moves_pattern = r'\b(?:[KQBNR]?[a-h]?[1-8]?[x-]?[a-h][1-8]|O-O(?:-O)?)\b'
    # Find all matches, strip spaces, and return as a list
    moves = re.findall(moves_pattern, user_input)
    return [move.strip() for move in moves if move.strip()]

In [5]:
interactive_chatbot(model, tokenizer, label_encoder)

Chatbot: Good day! I will try my best to predict chess moves based on your setting!
Type 'exit' to end the chat.
User: Predict the next move if the moves are e7, Nf6, c4, Nc6, g3, e6, and Nc3
Chatbot: Analyzing the moves: ['e7', 'Nf6', 'c4', 'Nc6', 'g3', 'e6', 'Nc3']
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 636ms/step
Chatbot: I can predict that the best next move is O-O.
User: exit
Chatbot: Thank you for chatting! Goodbye!
